In [1]:
import findspark
findspark.init('/home/mint/spark-2.1.0-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()
data = spark.read.csv('seeds_dataset.csv', inferSchema=True, header=True)
data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [2]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=data.columns, outputCol='features')
final_data = assembler.transform(data)

In [4]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')
final_data2 = scaler.fit(final_data).transform(final_data)

In [5]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans(featuresCol='scaledFeatures', k=3)
model = kmeans.fit(final_data2)
print('WSSE')
print(model.computeCost(final_data2))

WSSE
429.07559671506715


In [7]:
centers = model.clusterCenters()
import pprint
pprint.pprint(centers)

[array([ 4.87257659, 10.88120146, 37.27692543, 12.3410157 ,  8.55443412,
        1.81649011, 10.32998598]),
 array([ 6.31670546, 12.37109759, 37.39491396, 13.91155062,  9.748067  ,
        2.39849968, 12.2661748 ]),
 array([ 4.06105916, 10.13979506, 35.80536984, 11.82133095,  7.50395937,
        3.27184732, 10.42126018])]


In [9]:
model.transform(final_data2).select('prediction').show()

+----------+
|prediction|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         1|
|         1|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         2|
+----------+
only showing top 20 rows

